In [13]:
!pip install scikit-learn


  Using cached scikit_learn-1.5.1-cp39-cp39-win_amd64.whl.metadata (12 kB)
  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached threadpoolctl-3.5.0-py3-none-any.whl.metadata (13 kB)
   ---------------------------------------- 0.0/11.0 MB ? eta -:--:--
   -------- ------------------------------- 2.4/11.0 MB 12.2 MB/s eta 0:00:01
   ------------------ --------------------- 5.0/11.0 MB 12.1 MB/s eta 0:00:01
   --------------------------- ------------ 7.6/11.0 MB 12.4 MB/s eta 0:00:01
   ------------------------------------ --- 10.0/11.0 MB 11.9 MB/s eta 0:00:01
   ---------------------------------------- 11.0/11.0 MB 11.8 MB/s eta 0:00:00
Using cached joblib-1.4.2-py3-none-any.whl (301 kB)
Using cached threadpoolctl-3.5.0-py3-none-any.whl (18 kB)


In [2]:
import pandas as pd
usage_report = pd.read_csv(r'C:\Users\COMPUTER15\Desktop\caser_pytorch\datasets\edx\HMXPC13_DI_v2_5-14-14.csv')
embed_df = pd.read_csv(r'C:\Users\COMPUTER15\Desktop\caser_pytorch\datasets\edx\course_embeddings.csv')

In [9]:
merged_df = pd.merge(usage_report, embed_df, on='course_id')


In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Load the dataset
edx_data = pd.read_csv('datasets/edx/HMXPC13_DI_v2_5-14-14.csv')
course_embeddings = pd.read_csv('datasets/edx/course_embeddings.csv')

# Create a mapping from course_id to a unique integer
course_id_map = {cid: idx for idx, cid in enumerate(course_embeddings['course_id'].unique())}
course_name_map = {idx: name for idx, name in enumerate(course_embeddings['course_name'])}

# Create a mapping from user_id to a unique integer
user_id_map = {uid: idx for idx, uid in enumerate(edx_data['userid_DI'].unique())}

# Apply the mappings to the data, using a default value for unmapped entries
edx_data['user_id_mapped'] = edx_data['userid_DI'].map(user_id_map)
edx_data['course_id_mapped'] = edx_data['course_id'].map(course_id_map)

# Drop rows with NaN values in the mapped columns (if mapping fails)
edx_data.dropna(subset=['user_id_mapped', 'course_id_mapped'], inplace=True)

# Only keep relevant columns
edx_data = edx_data[['user_id_mapped', 'course_id_mapped', 'viewed']]

# Ensure 'viewed' column is binary (0 or 1)
edx_data['viewed'] = edx_data['viewed'].apply(lambda x: 1 if x > 0 else 0)

# Split the data into train, validation, and test sets
train, temp = train_test_split(edx_data, test_size=0.3, random_state=42)
val, test = train_test_split(temp, test_size=0.5, random_state=42)

# Save the train.txt file
train_file = 'datasets/edx/train.txt'
train[['user_id_mapped', 'course_id_mapped', 'viewed']].to_csv(train_file, sep=' ', header=False, index=False)

# Save the val.txt file
val_file = 'datasets/edx/val.txt'
val[['user_id_mapped', 'course_id_mapped', 'viewed']].to_csv(val_file, sep=' ', header=False, index=False)

# Save the test.txt file
test_file = 'datasets/edx/test.txt'
test[['user_id_mapped', 'course_id_mapped', 'viewed']].to_csv(test_file, sep=' ', header=False, index=False)

# Save the text.txt file (course id and name mapping)
text_file = 'datasets/edx/text.txt'
with open(text_file, 'w') as f:
    for idx, name in course_name_map.items():
        f.write(f"{idx} {name}\n")

print("Preprocessing complete. Files saved as train.txt, val.txt, test.txt, and text.txt.")


Preprocessing complete. Files saved as train.txt, val.txt, test.txt, and text.txt.


In [3]:
import pandas as pd
import numpy as np

# Load the course embeddings CSV file
course_embeddings_df = pd.read_csv('datasets/edx/course_embeddings.csv')

# Extract the embeddings (assuming the first two columns are course_id and course_name)
course_embeddings = course_embeddings_df.iloc[:, 2:].values  # Extracting the embeddings as a NumPy array

# Save the embeddings as a .npy file
np.save('datasets/edx/course_embeddings.npy', course_embeddings)

print("course_embeddings.npy file created successfully!")


course_embeddings.npy file created successfully!


In [1]:
import numpy as np
import pandas as pd
import os

# อ่านไฟล์ CSV
csv_path = 'datasets/edx/course_embeddings.csv'
df = pd.read_csv(csv_path)

# แยก course_id และ course_name ออกจาก embeddings
course_ids = df['course_id'].values
course_names = df['course_name'].values

# เลือกเฉพาะคอลัมน์ที่เป็น embeddings
embedding_columns = [col for col in df.columns if col.startswith('embed_')]
embeddings = df[embedding_columns].values

# บันทึก embeddings เป็นไฟล์ .npy
output_path = 'datasets/edx/precomputed_embeddings.npy'
np.save(output_path, embeddings)

print(f"Embeddings shape: {embeddings.shape}")
print(f"Saved precomputed embeddings to {output_path}")

# บันทึก course_ids และ course_names เป็นไฟล์ text เพื่อเก็บข้อมูลอ้างอิง
with open('datasets/edx/course_info.txt', 'w') as f:
    for id, name in zip(course_ids, course_names):
        f.write(f"{id}\t{name}\n")

print("Saved course info to datasets/edx/course_info.txt")

Embeddings shape: (16, 512)
Saved precomputed embeddings to datasets/edx/precomputed_embeddings.npy
Saved course info to datasets/edx/course_info.txt


In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedShuffleSplit

# Load the dataset
edx_data = pd.read_csv('datasets/edx/HMXPC13_DI_v2_5-14-14.csv')
course_embeddings = pd.read_csv('datasets/edx/course_embeddings.csv')

# Create a mapping from course_id to a unique integer
course_id_map = {cid: idx for idx, cid in enumerate(course_embeddings['course_id'].unique())}
course_name_map = {idx: name for idx, name in enumerate(course_embeddings['course_name'])}

# Create a mapping from user_id to a unique integer
user_id_map = {uid: idx for idx, uid in enumerate(edx_data['userid_DI'].unique())}

# Apply the mappings to the data, using a default value for unmapped entries
edx_data['user_id_mapped'] = edx_data['userid_DI'].map(user_id_map)
edx_data['course_id_mapped'] = edx_data['course_id'].map(course_id_map)

# Drop rows with NaN values in the mapped columns (if mapping fails)
edx_data.dropna(subset=['user_id_mapped', 'course_id_mapped'], inplace=True)

# Only keep relevant columns
edx_data = edx_data[['user_id_mapped', 'course_id_mapped', 'viewed']]

# Ensure 'viewed' column is binary (0 or 1)
edx_data['viewed'] = edx_data['viewed'].apply(lambda x: 1 if x > 0 else 0)

# Initialize StratifiedShuffleSplit
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.3, random_state=42)

# Split the data into train and temp sets using StratifiedShuffleSplit
for train_index, temp_index in sss.split(edx_data, edx_data['viewed']):
    train = edx_data.iloc[train_index]
    temp = edx_data.iloc[temp_index]

# Split the temp set into validation and test sets using StratifiedShuffleSplit
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.5, random_state=42)

for val_index, test_index in sss.split(temp, temp['viewed']):
    val = temp.iloc[val_index]
    test = temp.iloc[test_index]

# Save the train.txt file
train_file = 'datasets/edx/train.txt'
train[['user_id_mapped', 'course_id_mapped', 'viewed']].to_csv(train_file, sep=' ', header=False, index=False)

# Save the val.txt file
val_file = 'datasets/edx/val.txt'
val[['user_id_mapped', 'course_id_mapped', 'viewed']].to_csv(val_file, sep=' ', header=False, index=False)

# Save the test.txt file
test_file = 'datasets/edx/test.txt'
test[['user_id_mapped', 'course_id_mapped', 'viewed']].to_csv(test_file, sep=' ', header=False, index=False)

# Save the text.txt file (course id and name mapping)
text_file = 'datasets/edx/text.txt'
with open(text_file, 'w') as f:
    for idx, name in course_name_map.items():
        f.write(f"{idx} {name}\n")

print("Preprocessing complete. Files saved as train.txt, val.txt, test.txt, and text.txt.")


Preprocessing complete. Files saved as train.txt, val.txt, test.txt, and text.txt.
